In [1]:
%%configure -f
{
    "conf":  {       
             "spark.sql.catalog.glue_catalog1":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.glue_catalog1.warehouse":"s3://vasveena-dremio/iceberg/glue_catalog1/tables/",
             "spark.sql.catalog.glue_catalog1.catalog-impl":"org.apache.iceberg.aws.glue.GlueCatalog",
             "spark.sql.catalog.glue_catalog1.io-impl":"org.apache.iceberg.aws.s3.S3FileIO",
             "spark.sql.catalog.glue_catalog1.lock-impl":"org.apache.iceberg.aws.glue.DynamoLockManager",
             "spark.sql.catalog.glue_catalog1.lock.table":"myGlueLockTable",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
           } 
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1659608060840_0001,pyspark,idle,Link,Link,None,
3,application_1659608060840_0004,spark,busy,Link,Link,None,


In [2]:
%%sql 

use glue_catalog1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1659608060840_0005,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [3]:
%%sql
use streamdb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%sh 

aws s3 ls s3://vasveena-dremio/iceberg/glue_catalog1/tables/streamdb.db/streaming_trips_table/data/

2022-08-04 11:14:04       4121 00000-203-24321d45-c20a-407d-9c25-2414e2d26c63-00001.parquet
2022-08-04 11:14:33       5054 00000-407-8e8aa5f6-78aa-443b-89ac-d914d8911ed4-00001.parquet
2022-08-04 11:15:03       5518 00000-609-5f4e46bd-a5db-4b6b-9abe-b9562464ccbd-00001.parquet
2022-08-04 11:15:33       5734 00000-816-edfe8f73-d85f-4e82-9214-3c58f7a76aa9-00001.parquet
2022-08-04 11:14:04       4182 00002-225-44f72692-e6ff-4269-966d-cf2058501af2-00001.parquet
2022-08-04 11:14:33       5239 00002-429-ccb32a2e-34e2-414b-b990-902e33c5ca08-00001.parquet
2022-08-04 11:15:03       6005 00002-631-d025b0b2-38d6-4646-9bff-c71288d1a77f-00001.parquet
2022-08-04 11:15:33       5856 00002-838-309787ba-462f-45fe-bf95-a844e141c05a-00001.parquet
2022-08-04 11:14:04       4027 00003-236-70647d16-36ae-4bd4-8642-54b10c98b7be-00001.parquet
2022-08-04 11:14:33       5157 00003-440-a482b574-03d2-4c5f-bee8-d3aeb35f85d2-00001.parquet
2022-08-04 11:15:03       5486 00003-642-1b011262-fbbe-42ff-ac55-d6bb45536448-00

In [10]:
%%sql

select count(*) from glue_catalog1.streamdb.streaming_trips_table

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()